<a href="https://colab.research.google.com/github/alexlinapp/proofLLM/blob/main/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.6.0+cu124


In [22]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
query = inputs[1]
attention_scores = torch.empty(inputs.shape[0])
for i, xi in enumerate(inputs):
  attention_scores[i] = torch.dot(query, xi)
attention_scores, inputs.shape
attention_scores_normalized = attention_scores / torch.sum(attention_scores)
attention_scores_normalized, attention_scores, query.shape

(tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656]),
 tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]),
 torch.Size([3]))

In [3]:
context_vec_2 = torch.zeros(query.shape)
for i, xi in enumerate(inputs):
  context_vec_2 += attention_scores_normalized[i] * xi
  print(attention_scores_normalized[i].shape, xi.shape) # attention_scores_normalized is a vector containing the attention weights of each item imbedding
  # , xi is the actual input/item embedding
context_vec_2.shape


torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])
torch.Size([]) torch.Size([3])


torch.Size([3])

In [4]:
# compute all context vectors
atten_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    atten_scores[i, j] = torch.dot(x_i, x_j)
print(atten_scores)
# using for loops is too slow, notice we can use matrix multiplication
atten_score_matrix = inputs @ inputs.T
atten_score_matrix == atten_scores, atten_score_matrix
atten_weights = torch.softmax(atten_scores, dim=-1)
allcontext_vecs = atten_weights @ inputs
# To see this,
# --------------------
#   w1 = {w_{1,1}, w_{1,2}, ..., w_{1,n}} |   | a_1 = {a_{1,1}, a{1,2}, a_{1,3}}
#   w2 = .....                            |   | a_2 = {a{2,1}, ...}
#                                             |
allcontext_vecs

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [5]:
# Self-attention with trainable weights
x_2 = inputs[1];  # second input element
# print(x_2.shape)
print("x2: ", x_2)
d_in = inputs.shape[1]  # input embedding dimension
d_out = 2               # output embedding dimension
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))
print(W_query, W_value, W_key)
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
# print(query_2.shape, key_2.shape, value_2.shape)
print("qyery2: ", query_2)
keys = inputs @ W_key
values = inputs @ W_value
# print(keys.shape, values.shape)
keys2 = keys[1]
atten_score_22 = query_2.dot(keys2)#torch.dot(query_2, keys2)
# print("atten_score_22", atten_score_22)
atten_score_2 = query_2 @ keys.T
# print("query_2 shape: ", query_2.shape)
# print("keys shape: ", keys.shape)
print("atten_score_2: ", atten_score_2)
d_k = keys.shape[1]
atten_weights_2 = torch.softmax(atten_score_2 /  d_k**0.5, dim = -1) #  sqrt used to prevent sharp gradients during training
print(atten_weights_2)
context_vec_2 = atten_weights_2 @ values
print(context_vec_2)
# print(keys, keys2)

x2:  tensor([0.5500, 0.8700, 0.6600])
Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]], requires_grad=True) Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]], requires_grad=True) Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], requires_grad=True)
qyery2:  tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)
atten_score_2:  tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)
tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)
tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)


In [79]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
  def __init__(self, d_in, d_out) -> None:
    super().__init__()
    self.W_q = nn.Parameter(torch.rand(d_in, d_out))
    self.W_k = nn.Parameter(torch.rand(d_in, d_out))
    self.W_v = nn.Parameter(torch.rand(d_in, d_out))

  def forward(self, x):
    keys = x @ self.W_k
    values = x @ self.W_v
    queries = x @ self.W_q
    atten_score = queries @ keys.T
    attn_weights = torch.softmax(atten_score /  keys.shape[-1]**0.5, dim = -1)
    context_vec = attn_weights @ values
    return context_vec
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out) # d_in = 3 (embedding dimension), d_out = 2, output dimension
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [80]:
# using built in pytorch nn.Linear
class SelfAttention_v2(nn.Module):
  def __init__(self, d_in, d_out, qkv_bias=False) -> None:
    super().__init__()
    self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
  def forward(self, x):
    keys = self.W_k(x)
    values = self.W_v(x)
    queries = self.W_q(x)
    atten_scores = queries @ keys.T # done this way instead of V^T Q since want each row to correspond to same attention vector, which means each row needs same query
    atten_weights = torch.softmax(atten_scores /  keys.shape[-1]**0.5, dim = -1)
    context_vec = atten_weights @ values
    return context_vec
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
# print(sa_v2(inputs))
target_state_dict = sa_v1.state_dict()
print(sa_v1(inputs))
# for name, param in sa_v2.state_dict().items():
for name, param in target_state_dict.items():
  if name + ".weight" in sa_v2.state_dict():
    target_state_dict[name].copy_(sa_v2.state_dict()[name + ".weight"].T)
  # target_state_dict[name].copy_(param)
# sa_v2.load_state_dict(target_state_dict)
print(sa_v2(inputs))
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [95]:
queries = sa_v2.W_q(inputs)
keys = sa_v2.W_k(inputs)
values = sa_v2.W_v(inputs)
atten_scores = queries @ keys.T
atten_weights = torch.softmax(atten_scores /  keys.shape[-1]**0.5, dim = -1)
print(atten_weights)
# context_vec = atten_weights @ values
# print(context_vec)
context_length = atten_scores.shape[0]
mask = torch.tril(torch.ones(context_length, context_length))
print("mask: ", mask)
masked_simple = atten_weights * mask
print(masked_simple)
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
mask:  tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)
tensor([[1.0000, 0.0000,

In [108]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = atten_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)
atten_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim = -1)
print(atten_weights)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [116]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
all_ones = torch.ones(6, 6)
print(all_ones, dropout(all_ones))

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]) tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [118]:
torch.manual_seed(123)
print(dropout(atten_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [123]:
batch = torch.stack((inputs, inputs), dim=0); batch1 = torch.stack((inputs, inputs), dim=1)


In [153]:
class CasualAttention_v1(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False) -> None:
    super().__init__()
    self.d_out = d_out
    self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.Dropout = nn.Dropout(dropout)
    self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
  def forward(self, x):
    b, num_tokens, d_in = x.shape
    keys = self.W_k(x)
    values = self.W_v(x)
    queries = self.W_q(x)
    atten_scores = queries @ keys.transpose(1,2)  # batch matrix multiplication. Treat each batch element as a 2D matrix and then have a batch of resulting 2D matrices
    atten_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    atten_weights = torch.softmax(atten_scores /  keys.shape[-1]**0.5, dim = -1)
    atten_weights = self.Dropout(atten_weights)
    context_vec = atten_weights @ values
    return context_vec
# Wrapper class for multi-head attention. Just stacks CasualAttention_v1
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False) -> None:
    super().__init__()
    self.heads = nn.ModuleList([CasualAttention_v1(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)])
  def forward(self, x):
    return torch.cat([head(x) for head in self.heads], dim = -1)

In [168]:
torch.manual_seed(123)
d_in, d_out = 3, 2
context_length = batch.shape[1]
ca = CasualAttention_v1(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)
torch.manual_seed(123)
context_length = batch.shape[1] # number of tokens in a single context/batch element
d_in, d_out = 3, 1
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, 2)
print(mha(batch))
print(mha(batch).shape)
print(batch)


context_vecs.shape: torch.Size([2, 6, 2])
tensor([[[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]],

        [[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 2])
tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
